--- Day 16: Ticket Translation ---

As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

```
.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
```

Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

```
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
```

It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?


In [1]:
#!fsharp
let testInput="""class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
type Rule = seq<int*int>
type Rules = Map<string, Rule>
type Ticket = int array
type Task = { rules: Rules; ticket: Ticket; nearbyTickets: Ticket seq }

In [1]:
#!fsharp
let parseRules (input:string) =
    input.Split("\n")
    |> Seq.map
        (fun row ->
            let parts0 = row.Split(": ");
            let parts1 = parts0.[1].Split(" or ");
            (
                parts0.[0],
                parts1 
                |> Seq.map(
                    fun range ->
                        let parts2 = range.Split("-")
                        (int parts2.[0], int parts2.[1])
                )
            )
        ) 
    |> Map.ofSeq

let parseTickets heading (input:string) =
    input.Split("\n")
    |> 
    (fun rows -> 
        if rows.[0] <> heading then
            failwith ("Unexpected  heading " + rows.[0])
        else
            rows.[1..]
            |> Seq.map
                (fun ticket ->
                    ticket.Split(",")
                    |> Array.map int
                )
    )

let parse (input: string) = 
    input.Replace("\r","").Split("\n\n")
    |> 
    (fun blocks ->
        {
            rules = parseRules blocks.[0];
            ticket = parseTickets "your ticket:" blocks.[1] |> Seq.head;
            nearbyTickets = parseTickets "nearby tickets:" blocks.[2]
        }
    )

testInput |> parse

rules,ticket,nearbyTickets
"[ [class, Microsoft.FSharp.Collections.IEnumerator+mkSeq@132[System.Tuple`2[System.Int32,System.Int32]]], [row, Microsoft.FSharp.Collections.IEnumerator+mkSeq@132[System.Tuple`2[System.Int32,System.Int32]]], [seat, Microsoft.FSharp.Collections.IEnumerator+mkSeq@132[System.Tuple`2[System.Int32,System.Int32]]] ]","[ 7, 1, 14 ]","[ [ 7, 3, 47 ], [ 40, 4, 50 ], [ 55, 2, 20 ], [ 38, 6, 12 ] ]"


In [1]:
#!fsharp
let matchTicket (rules: Rules) (ticket: Ticket) =
    let matchEntry (entry:int) = 
        rules
        |> Map.toSeq
        |> Seq.map
            (fun (key, ranges) -> 
                Seq.tryFind (fun (rangeFrom, rangeTo) -> rangeFrom <= entry && entry <= rangeTo ) ranges
                |> Option.map (fun _ -> key)
            )
        |> Seq.choose id
        |> Set.ofSeq

    ticket
    |> Seq.map ( fun entry -> (entry, matchEntry entry) )

let testTask = testInput |> parse
matchTicket testTask.rules testTask.ticket

index,Item1,Item2
0,7,"[ class, row ]"
1,1,[ class ]
2,14,[ seat ]


In [1]:
#!fsharp
let errorRate (task: Task) =
    task.nearbyTickets
    |> Seq.map 
        ( fun ticket ->
            matchTicket task.rules ticket
            |> Seq.filter (fun (entry, rulenames) -> Set.isEmpty rulenames)
            |> Seq.map fst
        )
    |> Seq.concat
    |> Seq.sum

testInput |> parse |> errorRate

71

In [1]:
#!fsharp
taskInput |> parse |> errorRate

19087

--- Part Two ---

Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

```
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
```

Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?


In [1]:
#!fsharp
let testInput2 = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""

In [1]:
#!fsharp
let findMapping (task: Task) =
    let rec trimOptions (final: Map<string,int>) mapping =    
        let newExclusive = Seq.tryFind( fun (index,set) -> Set.count set = 1 ) mapping
        if newExclusive.IsSome then
            let value = fst newExclusive.Value
            let key = snd newExclusive.Value |> Set.toSeq |> Seq.head
            trimOptions 
                (final.Add( key, value) ) 
                (   mapping
                    |> Seq.map (
                         fun (index,set) ->
                            (index, set.Remove key)
                       )
                    |> Seq.filter ( 
                        fun (index, set) -> 
                            not (Set.isEmpty set) )
                )
        else
            final

    task.nearbyTickets
    |> Seq.map (matchTicket task.rules)
    |> Seq.map ( Seq.map snd )
    |> Seq.filter ((Seq.exists Set.isEmpty) >> (not))
    |> Seq.reduce 
        (fun acc ticket -> 
            acc 
            |> Seq.zip ticket
            |> Seq.map (fun (s1, s2) -> Set.intersect s1 s2)
        )
    |> Seq.toArray
    |> Array.mapi (fun index set -> (index,set) )
    |> trimOptions Map.empty

testInput2 |> parse |> findMapping

key,value
class,1
row,0
seat,2


In [1]:
#!fsharp
let findDepartures (task: Task) =
    findMapping task
    |> Map.toSeq
    |> Seq.fold (
        fun acc (key, index) ->
            if key.StartsWith("departure") then
                acc * int64(task.ticket.[index])
            else
                acc                
        )
        1L

taskInput |> parse |> findDepartures

1382443095281